In [3]:
import os
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, optimizers, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.metrics import Recall, Precision
# from keras.models import Model

In [1]:
test_path = 'data/chest_xray/test/'
train_path = 'data/chest_xray/train'
val_path = 'data/chest_xray/val/'

In [4]:
#Ensuring that it using my gpu and loading data using my gpu
with tf.device('/GPU:0'):

    test_generator = ImageDataGenerator(
        rescale=1.0/255).flow_from_directory(test_path, target_size=(64, 64), batch_size=312)
    train_generator = ImageDataGenerator(
        rescale=1.0/255).flow_from_directory(train_path, target_size=(64, 64), batch_size=2608)
    val_generator = ImageDataGenerator(
        rescale=1.0/255).flow_from_directory(val_path, target_size=(64, 64), batch_size=16)
    aug_train_generator = ImageDataGenerator(rescale=1.0/255,
                                             horizontal_flip=True,
                                             height_shift_range=0.2,
                                             width_shift_range=0.2,
                                             rotation_range=45,
                                             brightness_range=[0.2, 1],
                                             zoom_range=0.2
                                             ).flow_from_directory(train_path, target_size=(64, 64), batch_size=5000)

    #Grabbing data
    train_images, train_labels = next(train_generator)
    test_images, test_labels = next(test_generator)
    val_images, val_labels = next(val_generator)
    aug_train_images, aug_train_labels = next(aug_train_generator)

     # Create placeholders for train images
    train_images = tf.Variable(initial_value=train_images, shape=train_images.shape, trainable=False, dtype=tf.float32)
    test_images = tf.Variable(initial_value=test_images, shape=test_images.shape, trainable=False, dtype=tf.float32)
    val_images = tf.Variable(initial_value=val_images, shape=val_images.shape, trainable=False, dtype=tf.float32)
    aug_train_images = tf.Variable(initial_value=aug_train_images, shape=aug_train_images.shape, trainable=False, dtype=tf.float32)

    #Creating labels
    train_labels = tf.Variable(train_labels.T[0], trainable=False, dtype=tf.float32)
    test_labels = tf.Variable(test_labels.T[0], trainable=False, dtype=tf.float32)
    val_labels = tf.Variable(val_labels.T[0], trainable=False, dtype=tf.float32)
    aug_train_labels = tf.Variable(aug_train_labels.T[0], trainable=False, dtype=tf.float32)

    

Found 624 images belonging to 2 classes.
Found 867 images belonging to 1 classes.
Found 16 images belonging to 2 classes.
Found 867 images belonging to 1 classes.


In [ ]:
train_images.shape

In [ ]:
train_labels.shape

In [ ]:
plt.imshow(train_images[3])

In [ ]:
def evaluate(model):
    train_loss, train_acc = model.evaluate(train_images.numpy(), train_labels.numpy())
    test_loss, test_acc = model.evaluate(test_images.numpy(), test_labels.numpy())
    printout = pd.DataFrame({'Dataset' : pd.Series(dtype='str'), 
            'accuracy' : pd.Series(dtype='float64'), 
            'log_loss': pd.Series(dtype='float64')})
    test = ['Test', test_acc, test_loss]
    train = ['Train', train_acc, train_loss]
    printout.loc[len(printout.index)] = train
    printout.loc[len(printout.index)] = test
    return printout

In [ ]:
#try with only flat/dense
#FSM
model = models.Sequential()

model.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='sigmoid',
                        input_shape=(200, 200, 3)))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.summary()


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
fsm=model.fit(train_images.numpy(), train_labels.numpy(), initial_epoch=0, epochs = 25, batch_size = 64)

In [ ]:
evaluate(model)

In [ ]:
model2 = models.Sequential()

model2.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='sigmoid',
                        input_shape=(200, 200, 3)))

model2.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))

model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))

In [ ]:
learning_rates = [0.1, 0.01, 0.05]

with tf.device('/GPU:0'):
    for lr in learning_rates:
        #Setting learning rate
        optimizer = optimizers.Adam(learning_rate=lr)

        #compilew new model
        model2.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
        #fitting model
        model2.fit(train_images.numpy(), train_labels.numpy(), epochs=25, batch_size=64,
                   verbose=0)

        #evaluate models
        train_loss, train_accuracy = model2.evaluate(train_images.numpy(), train_labels.numpy(), batch_size = 64)
        test_loss, test_accuracy = model2.evaluate(test_images.numpy(), test_labels.numpy(), batch_size = 64)

        #printing an update
        print(f"Learning Rate: {lr}, Train Loss: {train_loss}, Train Accuracy: {train_accuracy}, Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


In [ ]:
# adding a early stopping

early_stopping = EarlyStopping(monitor='loss',
                               min_delta=1e-4,
                               verbose=1,
                               patience=5,
                               mode='min')


In [ ]:
model3 = models.Sequential()

model3.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='sigmoid',
                        input_shape=(200, 200, 3)))
model3.add(Dropout(.3))

model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Conv2D(64, (3, 3), activation='sigmoid'))
model3.add(Dropout(.3))

model3.add(MaxPooling2D(pool_size=(2, 2)))


model3.add(Flatten())
model3.add(Dense(12, activation='sigmoid'))
# model3.add(Dense(6, activation='sigmoid'))
model3.add(Dense(1, activation='sigmoid'))

model3.summary()


In [ ]:
optimizer = optimizers.Adam(learning_rate=.1)
model3.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

model3.fit(train_images.numpy(), train_labels.numpy(), epochs=25, batch_size=64, callbacks = [early_stopping])

In [ ]:
evaluate(model3)

In [ ]:
def createmodel(function_name='sigmoid', filters=32, dense=1, 
                regularization_type=None, regularization_strength=0.01):
    model = models.Sequential()

    if regularization_type == 'l1':
        model.add(Conv2D(filters=filters, kernel_size=(3, 3), activation=function_name, kernel_regularizer=regularizers.l1(regularization_strength)))
    elif regularization_type == 'l2':
        model.add(Conv2D(filters=filters, kernel_size=(3, 3), activation=function_name, kernel_regularizer=regularizers.l2(regularization_strength)))
    else:
        model.add(Conv2D(filters=filters, kernel_size=(
                      3, 3), activation=function_name))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(.3))
    model.add(Conv2D(filters=filters,
                     kernel_size=(3, 3),
                     activation=function_name))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(.3))
    model.add(Conv2D(filters=filters,
                     kernel_size=(3, 3),
                     activation=function_name))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    if dense == 3:
        model.add(Dense(12, activation = 'sigmoid'))
        model.add(Dense(6, activation='sigmoid'))
    elif dense == 2:
        model.add(Dense(6, activation = 'sigmoid'))
    model.add(Dense(1, activation='sigmoid'))


    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model


In [ ]:
model4 = createmodel('tanh', 64)
model4.fit(train_images.numpy(), train_labels.numpy(), epochs=25, batch_size=64)

In [ ]:
evaluate(model4)

In [ ]:
model5 = createmodel('relu', 128, 3)
model5.fit(train_images.numpy(), train_labels.numpy(), epochs=25, batch_size=64)
evaluate(model5)

In [ ]:
model6 = createmodel('relu', 128, 2)
model6.fit(train_images.numpy(), train_labels.numpy(), epochs=25, batch_size=64)
evaluate(model6)

In [ ]:
model7 = models.Sequential()

model7.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='sigmoid',
                        input_shape=(200, 200, 3)))
model7.add(Dropout(.3))

model7.add(MaxPooling2D(pool_size=(2, 2)))

model7.add(Conv2D(64, (3, 3), activation='sigmoid'))
model7.add(Dropout(.3))

model7.add(MaxPooling2D(pool_size=(2, 2)))


model7.add(Flatten())
model7.add(Dense(128, activation='sigmoid'))
model7.add(Dropout(.3))
model7.add(Dense(64, activation='sigmoid'))
model7.add(Dense(1, activation='sigmoid'))

model7.summary()

model7.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
with tf.device('/GPU:0'):
    model7.fit(train_images.numpy(), train_labels.numpy(), epochs=25, batch_size=64)
evaluate(model7)


In [ ]:
model8 = models.Sequential()

model8.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='tanh',
                        input_shape=(200, 200, 3)))
model8.add(Dropout(.3))

model8.add(MaxPooling2D(pool_size=(2, 2)))

model8.add(Conv2D(64, (3, 3), activation='tanh'))
model8.add(Dropout(.3))

model8.add(MaxPooling2D(pool_size=(2, 2)))


model8.add(Flatten())
model8.add(Dense(128, activation='tanh'))
model8.add(Dropout(.3))
model8.add(Dense(64, activation='tanh'))
model8.add(Dense(1, activation='sigmoid'))

model8.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
with tf.device('/GPU:0'):
    model8.fit(train_images.numpy(), train_labels.numpy(),
               epochs=25, batch_size=64)
    evaluate(model8)


In [ ]:
aug_model = models.Sequential()
aug_model.add(Conv2D(
    64, (3, 3), activation='relu', input_shape=(64, 64, 3)))
aug_model.add(MaxPooling2D(pool_size=(2, 2)))
aug_model.add(BatchNormalization())
aug_model.add(Dropout(0.8))

aug_model.add(Conv2D(128, (3, 3), activation='relu'))
aug_model.add(MaxPooling2D(pool_size=(2, 2)))
aug_model.add(BatchNormalization())

aug_model.add(Flatten())

aug_model.add(Dense(256, activation='relu'))
aug_model.add(BatchNormalization())

aug_model.add(Dense(1, activation='sigmoid',
                kernel_regularizer=regularizers.l2(0.001)))

aug_model.compile(optimizer=optimizers.Adam(learning_rate=.001),
                    loss='binary_crossentropy',
                    metrics=['accuracy', Recall(), Precision()])

In [ ]:
aug_model.fit(
    aug_train_images, aug_train_labels, epochs=25, batch_size=128)

In [ ]:
train_loss, train_acc, train_recall, train_precision = aug_model_8.evaluate(
    aug_train_images, aug_train_labels_final)
test_loss, test_acc, test_recall, test_precision = aug_model_8.evaluate(
    test_images, test_labels_final)
printout = pd.DataFrame({'Dataset': pd.Series(dtype='str'),
                         'accuracy': pd.Series(dtype='float64'),
                         'log_loss': pd.Series(dtype='float64'),
                         'precision': pd.Series(dtype='float64'),
                         'recall': pd.Series(dtype='float64')})
test = ['Test', test_acc, test_loss, test_recall, test_precision]
train = ['Train', train_acc, train_loss, train_recall, train_precision]
printout.loc[len(printout.index)] = train
printout.loc[len(printout.index)] = test
printout
